In [1]:
import pandas as pd
import numpy as np
import dropbox
import os
import matplotlib.pyplot as plt
# import altair as alt
# alt.renderers.enable('notebook')

In [2]:
# class Point():
#     def __init__(self, x, y):
#         self.x = x
#         self.y = y

#     def __repr__(self):
#         return 'Point({0},{1})'.format(self.x,self.y)

#     def translate(self, dx, dy):
#         "Add dx and dy to the point."
#         self.x += dx
#         self.y += dy

# e = Point(1.5, 2.5)

In [3]:
dbx = dropbox.Dropbox(os.environ['DROPBOX_API'])
dbx.users_get_current_account()

FullAccount(account_id='dbid:AABT1H-NJgK-ns_10QubSjssbwKMHtznqDU', name=Name(given_name='saurav', surname='chowdhury', familiar_name='saurav', display_name='saurav chowdhury', abbreviated_name='SC'), email='saurav13iitr@gmail.com', email_verified=True, disabled=False, locale='en', referral_link='https://www.dropbox.com/referrals/AADGzvUD1naruTWe92WGn9EaGcI-C2M2Zfg?src=app9-7501328', is_paired=False, account_type=AccountType('basic', None), root_info=UserRootInfo(root_namespace_id='884484377', home_namespace_id='884484377'), profile_photo_url=None, country='IN', team=None, team_member_id=None)

In [4]:
years = list(range(2002, 2020))
final_df = pd.DataFrame()

for year in years:
    path= "/Saurav/data/Stage_Specific_Ranking_-_Table_data"
    _, res = dbx.files_download(path+str(year)+".csv")
    df=pd.read_csv(res.raw)
    final_df = pd.concat([df,final_df])

final_df.shape

(21726, 114)

In [5]:
len(final_df['Athlete/Comp'].unique()) ## Unique ID for athele => Data at Athlete X Country X Competetion X Round lvl

21726

In [6]:
filtered_data = final_df.loc[:, ['Athlete/Comp','Metric', 'Score','Finish','Gender','Judge 1 Score','Judge 2 Score',
                          'Judge 3 Score','Judge 4 Score','Judge 5 Score','Judge 6 Score - 1','Judge 7 Score - 1',
                          'Round Group', 'Season','Season/Comp','Round','Wind']]

# filtered_data['age_yrs'] = 2020 - final_df['yob']
filtered_data['Athlete_name'] =  final_df['Athlete/Comp'].str.extract(r"(.*-....).-", expand=False)

filtered_data = filtered_data.reset_index(drop = True)

In [7]:
filtered_data.columns = filtered_data.columns.str.replace("/","_").str.replace("\ -\ ","_").str.replace("\ ","_")

In [8]:
filtered_data.columns

Index(['Athlete_Comp', 'Metric', 'Score', 'Finish', 'Gender', 'Judge_1_Score',
       'Judge_2_Score', 'Judge_3_Score', 'Judge_4_Score', 'Judge_5_Score',
       'Judge_6_Score_1', 'Judge_7_Score_1', 'Round_Group', 'Season',
       'Season_Comp', 'Round', 'Wind', 'Athlete_name'],
      dtype='object')

In [9]:
filtered_data[filtered_data['Metric'] != filtered_data['Score']].loc[:, 'Season'].unique()

array([2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004,
       2003, 2002], dtype=int64)

### Updating old scores in same scale as new metric


In [10]:

filtered_data['Metric'] = [round(filtered_data.loc[i,'Score']/30*100, 2) if filtered_data.loc[i, 'Metric'] != filtered_data.loc[i,'Score'] \
        else filtered_data.loc[i, 'Metric'] for i in range(len(filtered_data))]

filtered_data[filtered_data['Season'] == 2010].head()

,Athlete_Comp,Metric,Score,Finish,Gender,Judge_1_Score,Judge_2_Score,Judge_3_Score,Judge_4_Score,Judge_5_Score,Judge_6_Score_1,Judge_7_Score_1,Round_Group,Season,Season_Comp,Round,Wind,Athlete_name
13699,SMYSHLIAEV Aleksandr - OAR - World Cup 5 - 201...,52.77,15.83,15,Men,2.5,2.5,1.1,2.7,3.3,2.0,2.0,Finals,2010,2010-World Cup 5,Final 1,NaN,SMYSHLIAEV Aleksandr - OAR
13700,WILSON Bryon - USA - World Cup 7 - 2010 - Final 1,81.00,24.30,8,Men,4.3,4.6,4.5,4.5,4.4,2.0,2.0,Finals,2010,2010-World Cup 7,Final 1,0.0,WILSON Bryon - USA
13701,DUFOUR LAPOINTE Justine - CAN - North American...,68.87,20.66,6,Women,3.6,4.0,3.9,4.0,3.7,14.0,14.0,Qualification,2010,2010-North American Cup 2,Qualification 1,NaN,DUFOUR LAPOINTE Justine - CAN
13702,SALONEN Janne - FIN - World Cup 2 - 2010 - Qua...,69.20,20.76,39,Men,3.8,3.9,3.6,3.7,3.9,2.5,2.3,Qualification,2010,2010-World Cup 2,Qualification,4.0,SALONEN Janne - FIN
13703,KEATING Nicholas - USA - North American Cup 2 ...,69.87,20.96,19,Men,3.6,3.6,3.7,3.7,4.0,20.0,17.0,Qualification,2010,2010-North American Cup 2,Qualification 1,NaN,KEATING Nicholas - USA


### Adding rankings upto 30 for each competition

In [11]:
# Dropping the scores column
filtered_data = filtered_data.drop(columns = ['Score'])

#filtering the finals only
# filtered_data = filtered_data[filtered_data['Round'].str.lower().str.extract(r'.*(final).*', expand=False).notnull()]

print(filtered_data.shape)
#dividing competition data into 3 levels
tier1_df = filtered_data[filtered_data['Season_Comp'].str.lower().str.extract(r'.*(world).*', expand=False).notnull() |\
                 filtered_data['Season_Comp'].str.lower().str.extract(r'.*(olympics).*', expand=False).notnull()]

print(tier1_df.shape)

tier2_df = filtered_data[filtered_data['Season_Comp'].str.lower().str.extract(r'.*(north american).*', expand=False).notnull()]
print(tier2_df.shape)

tier3_df = filtered_data[filtered_data['Season_Comp'].str.lower().str.extract(r'.*(canad).*', expand=False).notnull()]
print(tier3_df.shape)

print(tier1_df.shape[0]+tier2_df.shape[0]+tier3_df.shape[0])

(21726, 17)
(15890, 17)
(4585, 17)
(1251, 17)
21726


In [104]:
# Dividing into men and women
tier1_df_M = tier1_df[tier1_df['Gender'] == "Men"]
tier1_df_W = tier1_df[tier1_df['Gender'] == "Women"]
tier2_df_M = tier2_df[tier2_df['Gender'] == "Men"]
tier2_df_W = tier2_df[tier2_df['Gender'] == "Women"]
tier3_df_M = tier3_df[tier3_df['Gender'] == "Men"]
tier3_df_W = tier3_df[tier3_df['Gender'] == "Women"]


In [105]:
## Correcting Datasets
temp_df = tier2_df_M[tier2_df_M['Athlete_Comp'] == 'CREAN Austen - CAN - North American Cup 3 - 2010 - Qualification 1']
tier2_df_M = tier2_df_M[tier2_df_M['Athlete_Comp'] != 'CREAN Austen - CAN - North American Cup 3 - 2010 - Qualification 1']
tier2_df_W = tier2_df_W.append(temp_df, ignore_index = True)
tier2_df_W.loc[tier2_df_W['Athlete_Comp'] == 'CREAN Austen - CAN - North American Cup 3 - 2010 - Qualification 1', 'Gender'] = "Women"

In [40]:
years = list(range(2002, 2020))
for year in years:
    comps = tier2_df_M[(tier2_df_M['Season'] == year)].loc[:, 'Season_Comp'].unique() 
    for comp in comps:
        print(comp)
        rounds = tier2_df_M[(tier2_df_M['Season'] == year) & (tier2_df_M['Season_Comp'] == comp)].loc[:, 'Round'].unique()
        print(sorted(rounds))

2010-North American Cup 2
['Final 1', 'Qualification 1']
2010-North American Cup 1
['Final 1']
2010-North American Cup 3
['Final 1', 'Qualification 1']
2010-North American Cup 4
['Final 1', 'Qualification 1']
2011-North American Cup 5
['Final 1', 'Qualification 1']
2011-North American Cup 3
['Final 1', 'Qualification 1']
2011-North American Cup 2
['Final 1', 'Qualification 1']
2011-North American Cup 1
['Final 1', 'Qualification 1']
2011-North American Cup 4
['Final 1', 'Qualification 1']
2012-North American Cup 4
['Final 1', 'Final 2', 'Qualification 1']
2012-North American Cup 6
['Final 1', 'Qualification 1']
2012-North American Cup 2
['Final 1', 'Final 2', 'Qualification 1']
2012-North American Cup 5
['Final 1', 'Final 2', 'Qualification 1']
2012-North American Cup 3
['Final 1', 'Final 2', 'Qualification 1']
2012-North American Cup 1
['Final 1', 'Final 2', 'Qualification 1']
2013-North American Cup 3
['Final 1', 'Final 2', 'Qualification 1']
2013-North American Cup 1
['Final 1', 'Qu

In [127]:
year = 2019
comp = "2019-World Championships"
# tier2_df_W[(tier2_df_W['Season'] == year) & (tier2_df_W['Season_Comp'] == comp) \
# & (tier2_df_W['Round'] == "Qualification 1") & (tier2_df_W['Finish'] == 22)]

rank_30_athletes(tier1_df_M, year, comp)

,Athlete_Comp,Metric,Finish,Gender,Judge_1_Score,Judge_2_Score,Judge_3_Score,Judge_4_Score,Judge_5_Score,Judge_6_Score_1,Judge_7_Score_1,Round_Group,Season,Season_Comp,Round,Wind,Athlete_name
0,KINGSBURY Mikael - CAN - World Championships -...,84.89,1,Men,18.0,17.9,18.2,18.5,18.0,7.4,8.0,Finals,2019,2019-World Championships,Final 2,0.0,KINGSBURY Mikael - CAN
1,GRAHAM Matt - AUS - World Championships - 2019...,81.94,2,Men,17.8,17.2,17.6,17.8,17.4,9.2,9.3,Finals,2019,2019-World Championships,Final 2,0.0,GRAHAM Matt - AUS
2,HARA Daichi - JPN - World Championships - 2019...,81.66,3,Men,17.0,16.8,16.9,16.4,16.2,8.5,8.8,Finals,2019,2019-World Championships,Final 2,0.0,HARA Daichi - JPN
3,HORISHIMA Ikuma - JPN - World Championships - ...,81.30,4,Men,17.4,17.1,17.1,17.6,17.2,8.2,8.3,Finals,2019,2019-World Championships,Final 2,0.0,HORISHIMA Ikuma - JPN
4,CAVET Benjamin - FRA - World Championships - 2...,81.02,5,Men,17.0,17.0,17.7,17.7,17.1,8.4,8.8,Finals,2019,2019-World Championships,Final 2,0.0,CAVET Benjamin - FRA
5,MARQUIS Philippe - CAN - World Championships -...,79.50,6,Men,17.4,16.9,16.7,16.8,16.6,7.8,8.1,Finals,2019,2019-World Championships,Final 2,0.0,MARQUIS Philippe - CAN
6,SALONEN Jimi - FIN - World Championships - 201...,78.90,7,Men,17.2,16.9,16.3,16.7,16.2,8.4,8.3,Finals,2019,2019-World Championships,Final 1,0.0,SALONEN Jimi - FIN
7,WALCZYK Dylan - USA - World Championships - 20...,76.73,8,Men,17.7,17.0,16.7,17.2,16.3,9.0,8.5,Finals,2019,2019-World Championships,Final 1,0.0,WALCZYK Dylan - USA
8,WALLBERG Walter - SWE - World Championships - ...,76.33,9,Men,16.8,17.3,16.9,17.0,17.0,7.2,7.1,Finals,2019,2019-World Championships,Final 1,0.0,WALLBERG Walter - SWE
9,SHIKATA Motoki - JPN - World Championships - 2...,75.12,10,Men,17.2,16.0,16.1,16.7,16.2,8.0,8.2,Finals,2019,2019-World Championships,Final 1,0.0,SHIKATA Motoki - JPN


In [124]:
# Function to rank all players MEN

def rank_30_athletes(data, year, comp):
    rounds = ['Qualification', 'Finals']
    
    temp_df = data[(data['Season'] == year) & (data['Season_Comp'] == comp)]
    
    q_rounds = temp_df[(temp_df['Round_Group'] == rounds[0])].loc[:, 'Round'].unique()
    q_rounds = sorted(q_rounds, reverse = True)
    
    f_rounds = temp_df[(temp_df['Round_Group'] == rounds[1])].loc[:, 'Round'].unique()
    f_rounds = sorted(f_rounds, reverse = True)
    
    # sorting the rounds in descending order of importance e.g final 1, qualifier 2, qualifier 1, ..
    rounds_desc_order = f_rounds + q_rounds
    
    output_df = temp_df[(temp_df['Round']== rounds_desc_order[0])].sort_values('Finish')
    ath_list = output_df['Athlete_name'].unique() # getting the list of athletes
    
    max_rank = max(output_df['Finish'])
    min_score = min(output_df['Metric'])
    prev_round = 1 #assuming there are more than 1 round
    drop_ranks = 0
    while max_rank < 30:
        
        if len(rounds_desc_order) <= prev_round: # checking if there is more than one round in the competition
            break
        
        # creating the dataframe for the rounds prior to the final
        next_ranks_df = temp_df[(temp_df['Round']== rounds_desc_order[prev_round])].sort_values('Finish')
        
        score_adj = next_ranks_df[next_ranks_df['Finish']== max_rank]['Metric'].values - \
                    next_ranks_df[next_ranks_df['Finish']== max_rank+1]['Metric'].values
        
        # adjusting scores for the next rounds
        new_score = min_score - score_adj
        
        # Filtering out the ranks already covered in the output dataframe
        next_ranks_df = next_ranks_df[next_ranks_df['Finish']> max_rank]

        for rank in next_ranks_df['Finish']:

            score_adj = next_ranks_df[next_ranks_df['Finish']== rank]['Metric'].values - \
                        next_ranks_df[next_ranks_df['Finish']== rank+1]['Metric'].values

            # checking if the athlete already exist in the previous round and removing it if it does
            if next_ranks_df.loc[next_ranks_df['Finish'] == rank, 'Athlete_name'].values not in ath_list:
                next_ranks_df.loc[next_ranks_df['Finish'] == rank, 'Metric'] = new_score
            else:
                next_ranks_df = next_ranks_df[next_ranks_df['Finish'] != rank]
                drop_ranks = drop_ranks+1 

            new_score = new_score - score_adj
        
        #adding the new ranks with adjusted scores to the output
        output_df = pd.concat([output_df, next_ranks_df], axis = 0, ignore_index = True)
        ath_list = output_df['Athlete_name'].unique() 
        max_rank = max(output_df['Finish'])

        if max_rank > 30:
            output_df = output_df[output_df['Finish'] <=30+drop_ranks]
            output_df.loc[:, 'Finish'] = np.arange(1,output_df.shape[0]+1) # aranging the ranks from 0 to 30
        else:
            prev_round = prev_round + 1
            output_df.loc[:, 'Finish'] = np.arange(1,output_df.shape[0]+1)
            
    return output_df[output_df['Finish']<=30]
          

In [125]:
# Function to rank all players WOMEN

def rank_20_athletes(data, year, comp):
    rounds = ['Qualification', 'Finals']
    
    temp_df = data[(data['Season'] == year) & (data['Season_Comp'] == comp)]
    
    q_rounds = temp_df[(temp_df['Round_Group'] == rounds[0])].loc[:, 'Round'].unique()
    q_rounds = sorted(q_rounds, reverse = True)
    
    f_rounds = temp_df[(temp_df['Round_Group'] == rounds[1])].loc[:, 'Round'].unique()
    f_rounds = sorted(f_rounds, reverse = True)
    
    # sorting the rounds in descending order of importance e.g final 1, qualifier 2, qualifier 1, ..
    rounds_desc_order = f_rounds + q_rounds
    
    output_df = temp_df[(temp_df['Round']== rounds_desc_order[0])].sort_values('Finish')
    ath_list = output_df['Athlete_name'].unique() # getting the list of athletes
    
    max_rank = max(output_df['Finish'])
    min_score = min(output_df['Metric'])
    prev_round = 1 #assuming there are more than 1 round
    drop_ranks = 0
    while max_rank < 20:
        
        if len(rounds_desc_order) <= prev_round: # checking if there is more than one round in the competition
            break
        
        # creating the dataframe for the rounds prior to the final
        next_ranks_df = temp_df[(temp_df['Round']== rounds_desc_order[prev_round])].sort_values('Finish')
        
        score_adj = next_ranks_df[next_ranks_df['Finish']== max_rank]['Metric'].values - \
                    next_ranks_df[next_ranks_df['Finish']== max_rank+1]['Metric'].values
        
        # adjusting scores for the next rounds
        new_score = min_score - score_adj
        
        # Filtering out the ranks already covered in the output dataframe
        next_ranks_df = next_ranks_df[next_ranks_df['Finish']> max_rank]

        for rank in next_ranks_df['Finish']:
            score_adj = next_ranks_df[next_ranks_df['Finish']== rank]['Metric'].values - \
                        next_ranks_df[next_ranks_df['Finish']== rank+1]['Metric'].values
            
            # checking if the athlete already exist in the previous round and removing it if it does
            if next_ranks_df.loc[next_ranks_df['Finish'] == rank, 'Athlete_name'].values not in ath_list:
                next_ranks_df.loc[next_ranks_df['Finish'] == rank, 'Metric'] = new_score
            else:
                next_ranks_df = next_ranks_df[next_ranks_df['Finish'] != rank]
                drop_ranks = drop_ranks+1 

            new_score = new_score - score_adj
        
        #adding the new ranks with adjusted scores to the output
        output_df = pd.concat([output_df, next_ranks_df], axis = 0, ignore_index = True)
        ath_list = output_df['Athlete_name'].unique() 
        max_rank = max(output_df['Finish'])
        
        if max_rank > 20:
            output_df = output_df[output_df['Finish'] <=20+drop_ranks]
            output_df.loc[:, 'Finish'] = np.arange(1,output_df.shape[0]+1) # aranging the ranks from 0 to 20
        else:
            prev_round = prev_round + 1
            output_df.loc[:, 'Finish'] = np.arange(1,output_df.shape[0]+1)
            
    return output_df[output_df['Finish']<=20]
          

In [128]:
def get_extended_ranks(data, func):
    
    years = list(range(2002, 2020))
    output_df = pd.DataFrame()
    
    for year in years:
        comps = data[(data['Season'] == year)].loc[:, 'Season_Comp'].unique()
        for comp in comps:
#             print(comp)
            out = func(data, year, comp)
            output_df = pd.concat([output_df, out], axis = 0, ignore_index=True)
    
    return output_df

In [129]:
tier1_df_M_updated_ranks = get_extended_ranks(tier1_df_M, rank_30_athletes)

In [130]:
tier2_df_M_updated_ranks = get_extended_ranks(tier2_df_M, rank_30_athletes)

In [131]:
tier3_df_M_updated_ranks = get_extended_ranks(tier3_df_M, rank_30_athletes)

In [132]:
tier1_df_W_updated_ranks = get_extended_ranks(tier1_df_W, rank_20_athletes)

In [133]:
tier2_df_W_updated_ranks = get_extended_ranks(tier2_df_W, rank_20_athletes)

In [134]:
tier3_df_W_updated_ranks = get_extended_ranks(tier3_df_W, rank_20_athletes)

In [135]:
tier1_df_M_updated_ranks = tier1_df_M_updated_ranks.rename(columns = {'Athlete_Comp' : 'Athlete_Comp_ID', 'Finish' : 'Finish_Rank', 'Metric' : 'Score_Metric'})
tier2_df_M_updated_ranks = tier2_df_M_updated_ranks.rename(columns = {'Athlete_Comp' : 'Athlete_Comp_ID', 'Finish' : 'Finish_Rank', 'Metric' : 'Score_Metric'})
tier3_df_M_updated_ranks = tier3_df_M_updated_ranks.rename(columns = {'Athlete_Comp' : 'Athlete_Comp_ID', 'Finish' : 'Finish_Rank', 'Metric' : 'Score_Metric'})

In [136]:
tier1_df_W_updated_ranks = tier1_df_W_updated_ranks.rename(columns = {'Athlete_Comp' : 'Athlete_Comp_ID', 'Finish' : 'Finish_Rank', 'Metric' : 'Score_Metric'})
tier2_df_W_updated_ranks = tier2_df_W_updated_ranks.rename(columns = {'Athlete_Comp' : 'Athlete_Comp_ID', 'Finish' : 'Finish_Rank', 'Metric' : 'Score_Metric'})
tier3_df_W_updated_ranks = tier3_df_W_updated_ranks.rename(columns = {'Athlete_Comp' : 'Athlete_Comp_ID', 'Finish' : 'Finish_Rank', 'Metric' : 'Score_Metric'})

In [137]:
tier1_df_M.to_csv("../data/Raw_tier_1_comp_data_men.csv", index = False)
tier2_df_M.to_csv("../data/Raw_tier_2_comp_data_men.csv", index = False)
tier3_df_M.to_csv("../data/Raw_tier_3_comp_data_men.csv", index = False)

tier1_df_W.to_csv("../data/Raw_tier_1_comp_data_women.csv", index = False)
tier2_df_W.to_csv("../data/Raw_tier_2_comp_data_women.csv", index = False)
tier3_df_W.to_csv("../data/Raw_tier_3_comp_data_women.csv", index = False)

In [138]:
tier1_df_M_updated_ranks.to_csv("../data/Updated_scores_tier_1_comp_data_men.csv", index = False)
tier2_df_M_updated_ranks.to_csv("../data/Updated_scores_tier_2_comp_data_men.csv", index = False)
tier3_df_M_updated_ranks.to_csv("../data/Updated_scores_tier_3_comp_data_men.csv", index = False)

tier1_df_W_updated_ranks.to_csv("../data/Updated_scores_tier_1_comp_data_women.csv", index = False)
tier2_df_W_updated_ranks.to_csv("../data/Updated_scores_tier_2_comp_data_women.csv", index = False)
tier3_df_W_updated_ranks.to_csv("../data/Updated_scores_tier_3_comp_data_women.csv", index = False)


In [139]:
files = ['Raw_tier_1_comp_data_men.csv', 'Raw_tier_2_comp_data_men.csv', 'Raw_tier_3_comp_data_men.csv',
         'Raw_tier_1_comp_data_women.csv', 'Raw_tier_2_comp_data_women.csv', 'Raw_tier_3_comp_data_women.csv',
         'Updated_scores_tier_1_comp_data_men.csv', 'Updated_scores_tier_2_comp_data_men.csv', 
         'Updated_scores_tier_3_comp_data_men.csv', 'Updated_scores_tier_1_comp_data_women.csv', 
         'Updated_scores_tier_2_comp_data_women.csv', 'Updated_scores_tier_3_comp_data_women.csv']

local_path = "../data/"
dest_path= "/Saurav/data/compiled_data/"

for file in files:
    filepath = local_path+file
    
    with open(filepath, "rb") as f:
       # we want to overwite any previous version of the file
        try:
            dbx.files_upload(f.read(), dest_path+file, mode=dropbox.files.WriteMode("overwrite"))
            print(file+" uploaded")
        except ApiError as err:
            # This checks for the specific error where a user doesn't have enough Dropbox space quota to upload this file
            if (err.error.is_path() and
                    err.error.get_path().error.is_insufficient_space()):
                sys.exit("ERROR: Cannot back up; insufficient space.")
            elif err.user_message_text:
                print(err.user_message_text)
                sys.exit()
            else:
                print(err)
                sys.exit()


Raw_tier_1_comp_data_men.csv uploaded
Raw_tier_2_comp_data_men.csv uploaded
Raw_tier_3_comp_data_men.csv uploaded
Raw_tier_1_comp_data_women.csv uploaded
Raw_tier_2_comp_data_women.csv uploaded
Raw_tier_3_comp_data_women.csv uploaded
Updated_scores_tier_1_comp_data_men.csv uploaded
Updated_scores_tier_2_comp_data_men.csv uploaded
Updated_scores_tier_3_comp_data_men.csv uploaded
Updated_scores_tier_1_comp_data_women.csv uploaded
Updated_scores_tier_2_comp_data_women.csv uploaded
Updated_scores_tier_3_comp_data_women.csv uploaded
